# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download abaixo da aula (se você puxar os dados direto do Kaggle pode ser que encontre resultados diferentes dos meus, afinal as bases de dados podem ter sido atualizadas).

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

### Importar Bibliotecas e Bases de Dados

In [17]:
import pandas as pd
import pathlib
from zipfile import ZipFile

In [68]:
caminho = pathlib.Path(r'../../PYTHON/HASHTAG/data/')

In [70]:
zipped_file = pathlib.Path(r'dataset - projeto ds.zip')

In [71]:
zip_file = ZipFile( caminho / zipped_file )
zip_file

<zipfile.ZipFile filename='..\\..\\PYTHON\\HASHTAG\\data\\dataset - projeto ds.zip' mode='r'>

In [76]:
zip_file.infolist()

[<ZipInfo filename='dataset/abril2018.csv' compress_type=deflate external_attr=0x20 file_size=123570840 compress_size=31084067>,
 <ZipInfo filename='dataset/abril2019.csv' compress_type=deflate external_attr=0x20 file_size=117472832 compress_size=29041237>,
 <ZipInfo filename='dataset/abril2020.csv' compress_type=deflate external_attr=0x20 file_size=117316975 compress_size=28936098>,
 <ZipInfo filename='dataset/agosto2018.csv' compress_type=deflate external_attr=0x20 file_size=119613599 compress_size=29911958>,
 <ZipInfo filename='dataset/agosto2019.csv' compress_type=deflate external_attr=0x20 file_size=114622228 compress_size=28387701>,
 <ZipInfo filename='dataset/dezembro2018.csv' compress_type=deflate external_attr=0x20 file_size=114032495 compress_size=28572587>,
 <ZipInfo filename='dataset/dezembro2019.csv' compress_type=deflate external_attr=0x20 file_size=114897257 compress_size=28465190>,
 <ZipInfo filename='dataset/fevereiro2019.csv' compress_type=deflate external_attr=0x20 f

In [75]:
base_airbnb = pd.DataFrame()
for i, text_file in enumerate(zip_file.infolist()):
    if (text_file.filename.endswith('.csv') and i == 1):
        print(i, text_file.filename)
        file = pd.read_csv(zip_file.open(text_file.filename))
        base_airbnb.append(file)
        

1 dataset/abril2019.csv


C:\Users\Caroline\AppData\Local\Temp\ipykernel_10328\3356911039.py:5: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv(zip_file.open(text_file.filename))
C:\Users\Caroline\AppData\Local\Temp\ipykernel_10328\3356911039.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_airbnb.append(file)


In [73]:
teste3 = pd.read_csv(zip_files[0])
teste3.head(3)

C:\Users\Caroline\AppData\Local\Temp\ipykernel_10328\1129210079.py:1: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  teste3 = pd.read_csv(zip_files[0])


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20190416181629,2019-04-16,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,none,This is the best spot in Rio. Everything happe...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,2.10
1,25026,https://www.airbnb.com/rooms/25026,20190416181629,2019-04-16,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",none,Copacabana is a lively neighborhood and the ap...,...,f,f,strict_14_with_grace_period,t,t,3,3,0,0,2.12
2,31560,https://www.airbnb.com/rooms/31560,20190416181629,2019-04-17,NICE & COZY 1BDR - IPANEMA BEACH,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,none,Die Nachbarschaft von Ipanema ist super lebend...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,2.33


### Consolidar Base de Dados

### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

### Tratar Valores Faltando

### Verificar Tipos de Dados em cada coluna

### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo